<a href="https://colab.research.google.com/github/Divina37055/pathon/blob/main/Copy_of_%EC%95%84%ED%8C%8C%EB%A7%A4%EA%B1%B0%EB%9E%98%EB%9F%89_%EC%A6%9D%EA%B0%80%EC%9C%A8%EC%9D%B4_%EB%86%92%EC%9D%80_%EC%A7%80%EC%97%AD_%EC%B0%BE%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta

# ==============================================================================
# 1. 유틸리티 함수 정의: 월 계산 및 컬럼명 형식 지정
# ==============================================================================

def get_month_string(year, month):
    """'YYYY년 M월' 형태의 문자열을 반환 (파일 컬럼명과 일치하도록 '0' 제거)."""
    # 1~9월에 '0'이 붙지 않도록 포맷팅을 수정했습니다.
    return f"{year}년 {month}월"

def get_previous_month(year, month):
    """입력된 년/월의 이전 달의 년/월을 계산하여 반환."""
    if month == 1:
        return year - 1, 12
    else:
        return year, month - 1

def get_analysis_months(target_year, target_month):
    """
    분석에 필요한 4개 월(타겟 월과 그 이전 3개월)의 년/월 튜플 리스트를 반환합니다.
    """
    month_list = []
    y, m = target_year, target_month

    # 총 4개의 월을 역순으로 저장합니다. (타겟 월부터 시작)
    for _ in range(4):
        month_list.append((y, m))
        y, m = get_previous_month(y, m)

    return month_list[::-1] # 가장 오래된 달부터 순서대로 정렬하여 반환

# ==============================================================================
# 2. 사용자 입력 받기
# ==============================================================================

print("## 📊 분석을 원하는 기준 월을 입력해 주세요.")
while True:
    try:
        input_year = int(input("분석 기준 년도 (YYYY): "))
        input_month = int(input("분석 기준 월 (MM): "))
        if 1 <= input_month <= 12:
            break
        else:
            print("❌ 월은 1부터 12 사이의 값으로 입력해주세요.")
    except ValueError:
        print("❌ 년도와 월을 숫자로 정확하게 입력해주세요.")

# 분석에 필요한 4개 월의 (년, 월) 튜플 리스트 생성
analysis_months_tuple = get_analysis_months(input_year, input_month)
# 컬럼명에 사용할 'YYYY년 M월' 문자열 리스트 생성
analysis_month_names = [get_month_string(y, m) for y, m in analysis_months_tuple]

print(f"\n✅ 분석 대상 기간: {analysis_month_names[0]}부터 {analysis_month_names[-1]}까지 (4개월 데이터 필요)")


# ==============================================================================
# 3. 파일 로드 (가장 최근의 성공 로직 사용)
# ==============================================================================

file_path_str = r'/content/(월) 행정구역별 아파트매매거래현황.csv'
file_path = Path(file_path_str)

df = None
load_successful = False

for encoding_type in ['cp949', 'utf-8']:
    try:
        df = pd.read_csv(file_path, encoding=encoding_type, header=0)
        load_successful = True
        # print(f"** 파일 로드 성공 ({encoding_type} 인코딩 적용).")
        break
    except Exception:
        pass

if not load_successful:
    print("\n[치명적 오류] 파일 로드 실패. 인코딩 및 파일 형식을 확인해주세요.")
    exit()

# ==============================================================================
# 4. 데이터 전처리 및 분석 기간 데이터 추출
# ==============================================================================

# 컬럼명 정리 (공백 -> 밑줄)
df.columns = df.columns.str.replace(r'\s+', '_', regex=True).str.strip()
trade_cols_underscore = [col.replace(' ', '_') for col in analysis_month_names]

REGION_COL_KEY = '지역통합'

# 필요한 컬럼만 선택
required_cols = [REGION_COL_KEY] + trade_cols_underscore
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"\n[치명적 오류] 분석에 필요한 월별 데이터가 부족합니다. 파일에 {missing_cols} 컬럼이 없습니다.")
    exit()

df_analysis = df[required_cols].copy()
df_analysis.columns = ['지역명'] + analysis_month_names # 컬럼명을 보기 좋게 복원
trade_months = analysis_month_names # 월별 컬럼 이름

# '전국' 행 및 결측값 제거
df_analysis = df_analysis[df_analysis['지역명'].notna()]
df_analysis = df_analysis[~df_analysis['지역명'].astype(str).str.contains('전국')]

# 거래건수 컬럼을 숫자형으로 변환 (쉼표 및 따옴표 제거 후 변환)
for col in trade_months:
    df_analysis[col] = df_analysis[col].astype(str).str.replace(r'["\s,]', '', regex=True)

df_analysis[trade_months] = df_analysis[trade_months].apply(pd.to_numeric, errors='coerce')


# ==============================================================================
# 5. 전월 대비 거래량 증가 비율 계산
# ==============================================================================

# 기간별 증가 비율 (Percentage Change) 계산
df_pct_change = df_analysis[trade_months].pct_change(axis=1)

# 비율 컬럼명 정의
pct_change_cols = []
for i in range(1, len(trade_months)):
    prev_month = trade_months[i-1]
    curr_month = trade_months[i]
    pct_change_col_name = f"{prev_month} 대비 {curr_month} 증가율 (%)"
    pct_change_cols.append(pct_change_col_name)

df_results = df_analysis[['지역명']].copy()
# 비율을 %로 변환하여 저장 (첫 번째 컬럼은 NaN이므로 1부터 시작)
df_results[pct_change_cols] = df_pct_change.iloc[:, 1:].values * 100

# ==============================================================================
# 6. 최종 출력: 가장 최근 월 기준 TOP 10 선정 후, 3개월 데이터 표시
# ==============================================================================

# 가장 최근 월의 증가율 컬럼을 기준으로 TOP 10 지역 선정
recent_pct_col = pct_change_cols[-1]
top_10_results = df_results.nlargest(10, recent_pct_col)

# 최종 출력 포맷팅
for col in pct_change_cols:
    top_10_results[col] = top_10_results[col].map('{:,.2f}%'.format)

print("\n" + "=" * 80)
print(f"## 🥇 {trade_months[-1]} 기준 전월 대비 증가율 TOP 10 지역 분석 결과")
print("-" * 80)
print(f"   (선정 기준: {recent_pct_col})")
print("=" * 80)

# 지역명을 인덱스로 설정하고 출력
print(top_10_results.set_index('지역명'))

print("\n" + "=" * 80)

## 📊 분석을 원하는 기준 월을 입력해 주세요.
분석 기준 년도 (YYYY): 2025
분석 기준 월 (MM): 
❌ 년도와 월을 숫자로 정확하게 입력해주세요.
분석 기준 년도 (YYYY): 2025
분석 기준 월 (MM): 4

✅ 분석 대상 기간: 2025년 1월부터 2025년 4월까지 (4개월 데이터 필요)

## 🥇 2025년 4월 기준 전월 대비 증가율 TOP 10 지역 분석 결과
--------------------------------------------------------------------------------
   (선정 기준: 2025년 3월 대비 2025년 4월 증가율 (%))
           2025년 1월 대비 2025년 2월 증가율 (%) 2025년 2월 대비 2025년 3월 증가율 (%)  \
지역명                                                                    
대구 군위군 군위군                         nan%                         nan%   
경북 영양군 영양군                         inf%                     -100.00%   
전남 진도군 진도군                       50.00%                       66.67%   
전북 무주군 무주군                       16.67%                      -71.43%   
경남 고성군 고성군                       20.00%                       50.00%   
전북 장수군 장수군                         inf%                      -66.67%   
경북 고령군 고령군                      -71.43%                      150.00%   
강원 홍천군 홍천